https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [ ]:
import os
import sys
import time
import json
import socket

from IPython.display import clear_output

In [ ]:
# request to get credentials at http://apps.twitter.com
consumer_key    = '**************************'
consumer_secret = '**************************'
access_token    = '**************************'
access_token_secret   = '**************************'

In [ ]:
import os
import re
import time
import json
import socket

import tweepy
from tweepy import Stream


# Documentation - https://docs.tweepy.org/en/stable/streaming.html
# Inherits from the Stream in tweepy - provides additional functionality
class TweetStream(Stream):
    
    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, client_connection, rate_limit_delay=None, time_limit=None, tweet_limit=None, message_Handler=None, batch_size=None, batch_handler=None):
        
        # Set client connection
        self.client_connection = client_connection

        # Set rate limt
        self.rate_limit_delay = rate_limit_delay

        # Set time limit
        self.start_time = time.time()
        self.time_limit = time_limit
        
        # Set tweet limit
        self.tweet_limit = tweet_limit
        self.tweet_count = 0

        # Set batching
        self.batch_size = batch_size
        self.batch_handler = batch_handler
        self.batch = []

        # Set message handler
        self.message_handler = message_Handler
        
        # Initialize super
        super(TweetStream, self).__init__(consumer_key, consumer_secret, access_token, access_token_secret)
        
    def check_continue_time_limit(self):
        
        # Check time limit if set
        if self.time_limit is not None:
            if (time.time() - self.start_time) > self.time_limit:
                return False
        return True
    
    def check_continue_tweet_limit(self):
        # Check tweet limit if set
        if self.tweet_limit is not None:
            if self.tweet_limit <= self.tweet_count :
                return False
        return True

    def close_connections(self):
        
        # Process batch before disconnecting
        self.process_batch(is_disconnecting=True)

        print("Disconnecting...")

        # Disconnect client connection if provided
        if self.client_connection is not None:
           self.client_connection.close()

        # Disconnect stream
        self.disconnect()

        print("Disconnected.")


    def send_to_client(self, data):

        if self.client_connection is not None:
            # Send tweet data to client connection
            self.client_connection.send(data)

            # Send new line delimiter to client connection
            self.client_connection.send(str('\n').encode('utf-8'))


    def process_batch(self, is_disconnecting=False):
        if self.batch_size is not None:
            if len(self.batch) == self.batch_size or (is_disconnecting == True and len(self.batch) > 0):
                # Write to batch handler if provided
                if self.batch_handler is not None:
                    self.batch_handler(self.batch)
            
                # Write to client connection if provided
                self.send_to_client(self.batch)

                # Clear batch
                self.batch.clear()


    def on_data(self, data):
        try:
            
            # Check for time limit
            if self.check_continue_time_limit() is False:
                print('Time limit hit.')
                self.close_connections()
                return False

            # Check for tweet limit
            if self.check_continue_tweet_limit() is False:
                print('Tweet limit hit.')
                self.close_connections()
                return False
            
            # Check for limit message from twitter
            if '{"limit":' in str(data):
                print('Twitter rate limit - {}'.format(str(data)))
                return True

            # Process batch
            if self.batch_size is not None:
                self.batch.append(data)
                self.process_batch()

            # Write to message handler if provided
            if self.message_handler is not None:
                self.message_handler(data)

            # Write to client connection if provided
            if self.client_connection is not None:
                self.send_to_client(data)

            # Update tweet count
            self.tweet_count = self.tweet_count + 1

            # Check for rate limit delay
            if self.rate_limit_delay is not None:
                time.sleep(self.rate_limit_delay)
                
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            self.close_connections()
            return False
        
        return True

    def if_error(self, status):
        print(status)
        return True

In [ ]:
def message_handler(data):
    clear_output(wait=True)
    print('Message Handler...')
    message = json.loads(data)
    print(message)

def batch_handler(batch):
    print('Batch Handler...')
    print(batch)
    
def send_tweets_to_client_connection(client_connection, topic):
    # Create twitter stream
    twitter_stream = TweetStream(\
        consumer_key\
        , consumer_secret\
        , access_token\
        , access_token_secret\
        , client_connection\
        , message_Handler=message_handler\
        , rate_limit_delay=1\
        #, tweet_limit=10\
        #, time_limit=5\
        #, batch_size=5\
        #, batch_handler=batch_handler\
        )
    
    # Filter for topic
    twitter_stream.filter(track=topic, languages=["en"])

In [ ]:
def start_client_connection_server(topic):
     # Get host name and port number for service.
     host = socket.gethostname()
     port = 5555
    
     # Initialize a socket
     s = socket.socket()
     s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
     # Binding host and port
     s.bind((host, port))

     print("Listening on port {} with topic {}:".format(port,topic))

     # Waiting for client connection
     s.listen(5)
    
     # Establish connection with client
     client_connection, addr = s.accept()  

     # Start streaming tweets to client
     with client_connection:
         print("Connected with client: " + str(addr))

         # Send tweets to client through the socket connection
         send_tweets_to_client_connection(client_connection, topic)

In [ ]:
if __name__ == "__main__":
    
    topic = 'batman'
    print("Topic = {}".format(topic))
    start_client_connection_server(topic)
    
#     # Get topic from args
#     args = sys.argv[1:]
#     if len(args) == 0:
#         # Start client connection server - with test data
#         topic = 'batman'
#         #print("Topic = {}".format(args))
#         start_client_connection_server(topic)
#         # time.sleep(10)
#         # print("Finished.")
#     else:
#         # Start client connection server
#         topic = args
#         #print("Topic = {}".format(args))
#         start_client_connection_server(topic)
#         # time.sleep(5)
#         # print("Finished.")